# SQL Server Assessment Tool

Performs a best-practices assessment on a local SQL Server Instance. A single assessment may take some time, so fill out the variables and execute the cell that matches the desired environment to perform the assessment needed.

## Notebook Variables

| Line | Variable | Description | Example |
| --- | --- | --- | --- |
| 1 | ServerInstance | Name of the SQL Server instance | MSSQLSERVER |
| 2 | OutputDb | New or existing database to place assessment results in | DB1 |
| 3 | OutputTable | Target table to place assessment results (for a clean assessment, use a new table name) | AssessmentResults |

In [ ]:
$ServerInstance = ""
$OutputDb       = ""
$OutputTable    = ""

## Notebook Steps

1. Ensure that the proper APIs and modules are installed per the <a href="..\prereqs.ipynb">prerequisites</a> notebook
2. Define above variables corresponding to the SQL Server instance to be assessed
3. Run the notebook to list the Assessment rules being considered, perform the assessment, and list the results
4. Fix any warnings and rerun Assessment API until clear by reviewing the results table

### Retrieve Checks for SQL Instance

Pipe the output of the Get-SqlInstance cmdlet to the _Get-SqlAssessmentItem_ cmdlet to get the list of checks and their status. These results ought to display a list of rules for a default assessment.

In [ ]:
Get-SqlInstance -ServerInstance $serverInstance | Get-SqlAssessmentItem

### Export Assessment to SQL Table

Invoke assessment for the instance and save the results to a SQL table by piping the output of the _Get-SqlInstance_ cmdlet to the _Invoke-SqlAssessment_ cmdlet. The results are piped to the _Write-SqlTableData_ cmdlet. The _Invoke-Assessment_ cmdlet is run with the -**FlattenOutput** parameter in this example. This parameter makes the output suitable for the _Write-SqlTableData_ cmdlet. The latter raises an error if the parameter is omitted.

In [ ]:
Get-SqlInstance -ServerInstance $serverInstance |
Invoke-SqlAssessment -FlattenOutput |
Write-SqlTableData -ServerInstance $serverInstance -DatabaseName $outputDb -SchemaName Assessment -TableName $OutputTable -Force

### Display the Results

Use _Invoke-SqlCmd_ cmdlet to execute a short query to display the assessment results in this notebook in a custom format.

In [ ]:
$q = "SELECT TOP (1000) * FROM [$outputDb].[Assessment].[$OutputTable]"
$results = Invoke-SqlCmd -Query $q -ServerInstance $ServerInstance
foreach ($result in $results)
{
    Write-Output "----------------------------------------------------------"
    Write-Output $result.CheckName
    if ($result.Severity -ne "Information")
    {
        Write-Warning $result.Message
    }
    else 
    {
        Write-Output $result.Message
    }
    Write-Output $result.HelpLink
}

## Next Steps

### Install SQL Assessment ADS Extension

Consider installing the **SQL Assessment** extension for Azure Data Studio from the extensions sidebar. Like this notebook, it uses the SQL Server Assessment API to provides a mechanism to evaluate the configuration of SQL Server for best practices. Simply search the Extensions Marketplace for "sql assessment" to list the extension package then click the green Install link to load it into ADS.